In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp
!pip install streamlit
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [ ]:
spark = sparknlp.start()

In [ ]:
### Select the model and re-run all the cells below ####

MODEL_NAME = 'wikiner_840B_300'

In [ ]:
## Generating Example Files ##

text_list = ["William Henry Gates III (né le 28 octobre 1955) est un magnat des affaires, développeur de logiciels, investisseur et philanthrope américain. Il est surtout connu comme le co-fondateur de Microsoft Corporation. Au cours de sa carrière chez Microsoft, Gates a occupé les postes de président, chef de la direction (PDG), président et architecte logiciel en chef, tout en étant le plus grand actionnaire individuel jusqu'en mai 2014. Il est l'un des entrepreneurs et pionniers les plus connus du révolution des micro-ordinateurs des années 1970 et 1980. Né et élevé à Seattle, Washington, Gates a cofondé Microsoft avec son ami d'enfance Paul Allen en 1975, à Albuquerque, au Nouveau-Mexique; il est devenu la plus grande société de logiciels informatiques au monde. Gates a dirigé l'entreprise en tant que président-directeur général jusqu'à sa démission en tant que PDG en janvier 2000, mais il est resté président et est devenu architecte logiciel en chef. À la fin des années 1990, Gates avait été critiqué pour ses tactiques commerciales, considérées comme anticoncurrentielles. Cette opinion a été confirmée par de nombreuses décisions de justice. En juin 2006, Gates a annoncé qu'il passerait à un poste à temps partiel chez Microsoft et à un emploi à temps plein à la Bill & Melinda Gates Foundation, la fondation caritative privée que lui et sa femme, Melinda Gates, ont créée en 2000. [ 9] Il a progressivement transféré ses fonctions à Ray Ozzie et Craig Mundie. Il a démissionné de son poste de président de Microsoft en février 2014 et a assumé un nouveau poste de conseiller technologique pour soutenir le nouveau PDG Satya Nadella.",
 "La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à Paris."]


In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# French ner is traing with glove_840B_300. So we use the same embeddings in the pipeline
embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', lang='xx').\
                    setInputCols(["document", 'token']).\
                    setOutputCol("embeddings")

public_ner = NerDLModel.pretrained(MODEL_NAME, 'fr') \
          .setInputCols(["document", "token", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverter() \
                .setInputCols(["document", "token", "ner"]) \
                  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[ documentAssembler, 
                                 tokenizer,
                                 embeddings,
                                 public_ner,
                                 ner_converter
                                 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)


+-------------------------------+---------+
|chunk                          |ner_label|
+-------------------------------+---------+
|William Henry Gates III        |PER      |
|Microsoft Corporation          |ORG      |
|Microsoft                      |ORG      |
|Gates                          |PER      |
|Né et élevé à Seattle          |MISC     |
|Washington                     |LOC      |
|Gates                          |PER      |
|Microsoft                      |ORG      |
|Paul Allen                     |PER      |
|Albuquerque                    |LOC      |
|Nouveau-Mexique                |LOC      |
|Gates                          |PER      |
|Gates                          |PER      |
|Gates                          |PER      |
|Microsoft                      |ORG      |
|Bill & Melinda Gates Foundation|ORG      |
|Melinda Gates                  |PER      |
|Ray Ozzie                      |PER      |
|Craig Mundie                   |PER      |
|Microsoft                      